# class drake::geometry::QueryObject< T >

In [1]:
# Import some basic libraries and functions for this tutorial.
import numpy as np
import os

from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import AddDefaultVisualization, ModelVisualizer
from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, SnoptSolver, IpoptSolver,  
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
    )

In [2]:
h = 1e-3
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=h)
parser = Parser(plant)

In [3]:
dir1 = "/Users/xiao/0_codes/ICBM_drake/models/objects"
dir2 = "/Users/xiao/0_codes/ICBM_drake/models/ycb/sdf/"
table_file = os.path.join(dir1, "table_top.sdf")
panda_file = "/Users/xiao/0_codes/ICBM_drake/models/franka_description/urdf/panda_arm_hand.urdf"
box_file = os.path.join(dir2, "003_cracker_box.sdf")
table_modelInstance = parser.AddModels(table_file)[0]  # this return pydrake.multibody.tree.ModelInstanceIndex
panda_modelInstance = parser.AddModels(panda_file)[0]
box_modelInstance = parser.AddModels(box_file)[0]

table_top_frame = plant.GetFrameByName("table_top_center")
robot_base = plant.GetFrameByName("panda_link0")

X_W_table = RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],]),
  p=[0.0, 0.0, 0.0]
)

plant.WeldFrames(plant.world_frame(), table_top_frame, X_W_table)
plant.WeldFrames(table_top_frame, robot_base, RigidTransform(RotationMatrix.Identity(), [0., -0.4, 0.]))
plant.Finalize()
plant.set_name("table_with_box")
diagram = builder.Build()
context = diagram.CreateDefaultContext()
mutable_context = plant.GetMyContextFromRoot(context)

<div class="alert alert-block alert-success">
<b>Querying the scene geometry based on context at $t_k$:</b> 1. create mutable_contexts from root_context for each time step; 2. at $t_k$, using its corresponding mutable_context to create a query_port; 3. create query_object, which has some useful member functions; 4. create the inspector = query_object.inspector() to extract informations from the scene geometries.
</div>

In [5]:
query_port = plant.get_geometry_query_input_port()

In [9]:
if query_port.HasValue(mutable_context):
    query_object = query_port.Eval(mutable_context)

In [10]:
signed_distance_pairs = query_object.ComputeSignedDistancePairwiseClosestPoints()

In [26]:
print(len(signed_distance_pairs))

2165


In [16]:
print(signed_distance_pairs[0].id_A)
print(signed_distance_pairs[0].id_B)

<GeometryId value=21>
<GeometryId value=282>


In [23]:
# using the inspector to get info on the pairs
inspector = query_object.inspector()
frameA_id = inspector.GetFrameId(signed_distance_pairs[0].id_A)
frameB_id = inspector.GetFrameId(signed_distance_pairs[1].id_B)

bodyA = plant.GetBodyFromFrameId(frameA_id)
bodyB = plant.GetBodyFromFrameId(frameB_id)

In [24]:
type(bodyA)

pydrake.multibody.tree.RigidBody

In [27]:
print(bodyA)
print(bodyB)

<RigidBody name='table_top_link' index=1 model_instance=2>
<RigidBody name='base_link_cracker' index=14 model_instance=4>


__A variant of ComputeSignedDistancePairwiseClosestPoints() which computes the signed distance (and witnesses) between a specific pair of geometries indicated by id.__ 

In [28]:
"""
# This could be used for getting pairs between finger and box
query_object.ComputeSignedDistancePairClosestPoints(GeometryId geometry_id_A,
                                                    GeometryId geometry_id_B)
"""                                                     """
# maybe use this 
query_object.ComputeSignedDistanceToPoint()
"""

# get link 7 in {W}
link7_frame = plant.GetFrameByName("panda_link7")

In [30]:
print(link7_frame)

<BodyFrame name='panda_link7' index=11 model_instance=3>


In [35]:
link7_frame.CalcPoseInBodyFrame(mutable_context)

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)

In [37]:
link7_frame.CalcPoseInWorld(mutable_context)

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, -1.0, -9.793177720293495e-12],
    [0.0, 9.793177720293495e-12, -1.0],
  ]),
  p=[0.088, -0.39999999999966707, 1.033],
)

<div class="alert alert-block alert-success">
<b>I know what to do next!!:</b> Manually choose a point on the finger in {link8}. Then manually choose three points of the box to give surface normal. Something useful relevant to contact constraints: https://drake.mit.edu/doxygen_cxx/group__contact__surface__constraints.html
</div>

__this will be useful!__ #include "drake/geometry/proximity/calc_distance_to_surface_mesh.h" and "proximity_utilities.h" These are what I will be doing manually.

In [41]:
box_frame = plant.GetFrameByName("base_link_cracker")
print(f"box fram in W: {box_frame.CalcPoseInWorld(mutable_context)}")

box fram in W: RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)


In [43]:
panda_modelInstance

ModelInstanceIndex(3)

In [44]:
plant.world_frame()

<BodyFrame name='world' index=0 model_instance=0>

In [47]:
plant.world_frame().CalcPoseInWorld(mutable_context)

RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)

In [301]:
""" a function to extract collision points from the box
    These points are expressed in {box}
"""
def getPointsFromCollisionBox(modelfile):
    points = []
    with open(modelfile, 'r') as f:
        model_file = f.read()
    model_data = bs(model_file, "xml")
    box_collision_points = model_data.find_all('collision') # this gathers all <collision> tags in the model.
    # print(box_collision_points)
    for ele in box_collision_points:
        if ele["name"].split('_')[0] == "point":
            coords = ele.text.strip().split(' ')
            points.append([coords[0], coords[1], coords[2]])
    return np.array(points).astype(float)
    
points = getPointsFromCollisionBox(box_file)

__Now I can compute the surfaces normal, then project a point in hand onto the surface.__
__Next up: get a point in left finger, then express in {W}__
__Then, compute the relative velocity at contact point {C}__

In [331]:
# get the transformation from {rightfinger} to {link7}
rightfinger_frame = plant.GetFrameByName("panda_rightfinger")
link7_frame = plant.GetFrameByName("panda_link7")
X_w_rightfinger = rightfinger_frame.CalcPoseInWorld(mutable_context)
X_w_link7 = link7_frame.CalcPoseInWorld(mutable_context)
X_link7_p = X_w_link7.inverse().multiply(X_w_rightfinger)

# from the urdf collision bodies get a roughly exstimated points, express in {link7}
p_rf = np.array([[0], [-0.007], [0.045]])
p_link7_rf = X_link7_p.multiply(p_rf)
print(f"A point of the right finger expressed in panda_link7 is:\n {p_link7_rf}")

A point of the right finger expressed in panda_link7 is:
 [[-0.00494975]
 [-0.00494975]
 [ 0.2104    ]]


In [361]:
frameIds = inspector.GetAllFrameIds()
# plant.GetBodyFromFrameId(frameIds)
def getFrameIdByLinkName(linkName, frameIds):
    for id in frameIds:
        if plant.GetBodyFromFrameId(id).name() == linkName:
            return id

<div class="alert alert-block alert-danger">
<b>Just realized!!:</b> I should not rigidly follow the C++ sliding_friction_complementarity_constraint.cc line 211-216. Because in Drake's pipeline, the contact points are detected by SceneGraph(), which are registered as points in the colliding geometries. Then it makes sense to transform the points into body frame. I can just do this manually as I know the contacts in bodies and their poses. 
</div>

# Continue here!

In [405]:
p_in_geom_finger = np.array([[0], [-0.007], [0.045]])
rightfinger_frameId = getFrameIdByLinkName("panda_rightfinger", frameIds)
rightfinger_pose = rightfinger_frame.CalcPoseInBodyFrame(mutable_context)
p_in_rightfinger_frame = rightfinger_pose @ p_in_geom_finger
p_in_rightfinger_frame

array([[ 0.   ],
       [-0.007],
       [ 0.045]])

In [382]:
frameIds

[<FrameId value=9>,
 <FrameId value=17>,
 <FrameId value=23>,
 <FrameId value=69>,
 <FrameId value=91>,
 <FrameId value=113>,
 <FrameId value=132>,
 <FrameId value=157>,
 <FrameId value=197>,
 <FrameId value=216>,
 <FrameId value=226>,
 <FrameId value=230>,
 <FrameId value=246>,
 <FrameId value=256>,
 <FrameId value=268>]

In [385]:
plant.GetBodyFromFrameId(rightfinger_frameId).body_frame().CalcPoseInWorld(mutable_context)

RigidTransform(
  R=RotationMatrix([
    [0.7071067811868645, 0.7071067811862305, 0.0],
    [0.7071067811862305, -0.7071067811868645, -9.793177720293495e-12],
    [-6.92482237538144e-12, 6.924822375387649e-12, -1.0],
  ]),
  p=[0.088, -0.4000000000012869, 0.8675999999999999],
)

In [387]:
plant.GetBodyByName("panda_rightfinger").body_frame().CalcPoseInWorld(mutable_context)

RigidTransform(
  R=RotationMatrix([
    [0.7071067811868645, 0.7071067811862305, 0.0],
    [0.7071067811862305, -0.7071067811868645, -9.793177720293495e-12],
    [-6.92482237538144e-12, 6.924822375387649e-12, -1.0],
  ]),
  p=[0.088, -0.4000000000012869, 0.8675999999999999],
)

In [398]:
rf_frame = plant.GetBodyByName("panda_rightfinger").body_frame()
rf_frame.CalcPoseInWorld(mutable_context)

for geoid in geoIds:
    if inspector.GetPoseInFrame(geoid) == rf_frame.CalcPoseInBodyFrame(mutable_context):
        print(geoid)

In [400]:
inspector.GetPoseInFrame(geoIds[28])


RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [-0.0, 0.0, 1.0],
  ]),
  p=[0.0, -0.171, 0.0],
)

In [371]:
rightfinger_frame = plant.GetFrameByName("panda_rightfinger")
rightfinger_frame.CalcPoseInWorld(mutable_context)

RigidTransform(
  R=RotationMatrix([
    [0.7071067811868645, 0.7071067811862305, 0.0],
    [0.7071067811862305, -0.7071067811868645, -9.793177720293495e-12],
    [-6.92482237538144e-12, 6.924822375387649e-12, -1.0],
  ]),
  p=[0.088, -0.4000000000012869, 0.8675999999999999],
)

In [ ]:
# get spatial velocity of the point p in {link7}
V_rf_box_w = rightfinger_frame.CalcSpatialVelocity(mutable_context, box_frame, plant.world_frame())

# get witness point in {w}
p_link7_rf_W = plant.CalcRelativeTransform(mutable_context, plant.world_frame(), link7_frame) @ p_link7_rf;
print(p_link7_rf_W)

In [329]:
type(rightfinger_frame.index())

pydrake.multibody.tree.FrameIndex

In [320]:
dir(rightfinger_frame)

['CalcAngularVelocity',
 'CalcOffsetPoseInBody',
 'CalcOffsetRotationMatrixInBody',
 'CalcPose',
 'CalcPoseInBodyFrame',
 'CalcPoseInWorld',
 'CalcRelativeSpatialAcceleration',
 'CalcRelativeSpatialAccelerationInWorld',
 'CalcRelativeSpatialVelocity',
 'CalcRelativeSpatialVelocityInWorld',
 'CalcRotationMatrix',
 'CalcRotationMatrixInBodyFrame',
 'CalcRotationMatrixInWorld',
 'CalcSpatialAcceleration',
 'CalcSpatialAccelerationInWorld',
 'CalcSpatialVelocity',
 'CalcSpatialVelocityInWorld',
 'EvalAngularVelocityInWorld',
 'GetFixedOffsetPoseInBody',
 'GetFixedPoseInBodyFrame',
 'GetFixedRotationMatrixInBody',
 'GetFixedRotationMatrixInBodyFrame',
 'GetParentPlant',
 '__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__

In [340]:
dir(box_modelInstance)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_pybind11_del_orig',
 'is_valid']